# Agentes com Autogen - ReAct - Hands-On

Agradecimento ao professor Julio Cesar dos Reis pelo material.

## Descrição

Exploramos a implementação de agentes simples utilizando a biblioteca Autogen. O objetivo é demonstrar como explorar o ReAct.


In [1]:
!pip install autogen
!pip install --upgrade pydantic
!pip install ipywidgets
!pip install groq
!pip install "autogen-agentchat~=0.2" "tavily-python"
!pip install autogen-agentchat[teachable]~=0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.5/787.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.4
    Uninstalling pydantic-2.11.4:
      Successfully uninstalled pydantic-2.11.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
 

In [2]:
import os
from typing import Literal
from typing import Annotated
from tavily import TavilyClient
import autogen
from typing_extensions import Annotated
import getpass

API key in https://docs.tavily.com/welcome

In [ ]:
# Solicitar a API key de forma segura
api_key = getpass.getpass("Enter your Groq API key: ")
tavily_key = getpass.getpass("Enter your Tavily API key: ")

config_list = [
    {
      "model": "llama3-8b-8192",
      "api_key": api_key,
      "api_type": "groq"
    }
]

Tavily is a search engine optimized for LLMs, aimed at efficient, quick and persistent search results.

In [ ]:
llm_config = {
    "config_list": config_list,
    "timeout": 120
}

tavily = TavilyClient(api_key=tavily_key)

def search_tool(query: Annotated[str, "The search query"]) -> Annotated[str, "The search results"]:
    return tavily.get_search_context(query=query, search_depth="advanced")


Construct your ReAct prompt

In [ ]:
# NOTE: this ReAct prompt is adapted from Langchain's ReAct agent: https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/react/agent.py#L79
ReAct_prompt = """
Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
"""

# Define the ReAct prompt message. Assuming a "question" field is present in the context


def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

In [ ]:
from autogen.agentchat.contrib.capabilities import teachability
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor
from autogen import register_function

# Setting up code executor.
os.makedirs("coding", exist_ok=True)
# Use docker executor for running code in a container if you have docker installed.
# code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
code_executor = LocalCommandLineCodeExecutor(work_dir="coding")

user_proxy = autogen.UserProxyAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER", #ALWAYS
    max_consecutive_auto_reply=5,
    code_execution_config={"executor": code_executor},
)

assistant = autogen.AssistantAgent(
    name="Assistant",
    system_message="Only use the tools you have been provided with. Reply TERMINATE when the task is done.",
    llm_config={"config_list": config_list, "cache_seed": None},
)

# Register the search tool.
register_function(
    search_tool,
    caller=assistant,
    executor=user_proxy,
    name="search_tool",
    description="Search the web for the given query",
)

# Cache LLM responses. To get different responses, change the cache_seed value.
with Cache.disk(cache_seed=43) as cache:
    user_proxy.initiate_chat(
        assistant,
        message=react_prompt_message,
        question="Who is the best support hero In marvel rivals?",
        cache=cache,
    )

ReAct with memory module enabled via teachability
Gives to an agent the ability to remember user teachings

https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/contrib/capabilities/teachability/

In [ ]:
#!pip install "autogen-agentchat[teachable]~=0.2"
#!pip install ag2[teachable]
!pip install "autogen-agentchat[teachable]~=0.2"

In [ ]:
from autogen.agentchat.contrib.capabilities.teachability import Teachability

# Instantiate the Teachability capability. Its parameters are all optional.
teachability = teachability.Teachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="./tmp/notebook/teachability_db",
    recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

# Now add the Teachability capability to the agent.
teachability.add_to_agent(assistant)

with Cache.disk(cache_seed=43) as cache:
    user_proxy.initiate_chat(
        assistant,
        message=react_prompt_message,
        question="Who is the best support hero In marvel rivals?",
        cache=cache,
    )